In [2]:
from smolagents import tool, CodeAgent, LiteLLMModel
import requests, json
from bs4 import BeautifulSoup
from huggingface_hub import HfApi
import arxiv
from pypdf import PdfReader

# 1️⃣ Mehrere Paper holen
@tool
def get_top_three_papers() -> list:
    """
    Returns a list of the top 3 daily papers (titles) from Hugging Face.
    """
    url = "https://huggingface.co/papers"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    containers = soup.find_all('div', class_='SVELTE_HYDRATER contents')
    titles = []

    for container in containers:
        data_props = container.get('data-props', '')
        if not data_props:
            continue
        try:
            data_json = json.loads(data_props.replace('&quot;', '"'))
            if 'dailyPapers' in data_json:
                for paper in data_json['dailyPapers'][:3]:
                    titles.append(paper['title'])
                break
        except json.JSONDecodeError:
            continue

    return titles or ["No papers found."]


# 2️⃣ ID finden
@tool
def get_paper_id_by_title(title: str) -> str:
    """
    Returns the arXiv paper ID by its title.
    Args:
        title: The paper title for which to get the ID.
    """
    api = HfApi()
    papers = api.list_papers(query=title)
    if papers:
        paper = next(iter(papers))
        return paper.id
    else:
        return "No paper ID found."


# 3️⃣ Paper herunterladen
@tool
def download_paper_by_id(paper_id: str) -> str:
    """
    Downloads the arXiv paper by ID and saves it as 'paper_<id>.pdf'.

    Args:
        paper_id: The arXiv ID of the paper to download (e.g., "1706.03762").
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(id_list=[paper_id])
        paper = next(client.results(search))
        filename = f"paper_{paper_id}.pdf"
        paper.download_pdf(filename=filename)
        return filename
    except Exception as e:
        return f"Error downloading paper: {e}"


# 4️⃣ PDF lesen
@tool
def read_pdf_file(file_path: str) -> str:
    """
    Reads the first 3 pages of a PDF and returns text.
    Args:
        file_path: Path to the PDF file.
    """
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages[:3]:
            text += page.extract_text() or ""
        return text.strip()
    except Exception as e:
        return f"Error reading {file_path}: {e}"


# 5️⃣ Agent definieren
model = LiteLLMModel(model_id="ollama_chat/glm-4.6:cloud")

agent = CodeAgent(
    tools=[get_top_three_papers, get_paper_id_by_title, download_paper_by_id, read_pdf_file],
    model=model,
    stream_outputs=True
)

# 6️⃣ Ziel definieren
agent.run("""
Fetch the top 3 papers from Hugging Face daily papers.
For each paper:
- get its arXiv ID,
- download it,
- read the first pages,
- and summarize it.
Then create a short consolidated summary comparing the three papers.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Fetch the top 3 papers from Hugging Face daily papers.                                                          │
│ For each paper:                                                                                                 │
│ - get its arXiv ID,                                                                                             │
│ - download it,                                                                                                  │
│ - read the first pages,                                                                                         │
│ - and summarize it.                                                                                             │
│ Then create a short consolidated summary comparing the three papers.                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/glm-4.6:cloud ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/opt/anaconda3/envs/trans/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_papers = get_top_three_papers()                                                                              
  print("Top 3 papers:")                                                                                           
  for i, paper in enumerate(top_papers, 1):                                                                        
      print(f"{i}. {paper}")                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Top 3 papers:
1. JanusCoder: Towards a Foundational Visual-Programmatic Interface for
  Code Intelligence
2. Video-Thinker: Sparking "Thinking with Videos" via Reinforcement
  Learning
3. ReForm: Reflective Autoformalization with Prospective Bounded Sequence
  Optimization

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(final_data)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'papers': [{'title': 'JanusCoder: Towards a Foundational Visual-Programmatic Interface for Code 
Intelligence', 'id': '2510.23538', 'summary': 'JanusCoder: Towards a Foundational Visual-Programmatic Interface for
Code Intelligence\nThis paper introduces JanusCoder, a system that combines visual and programmatic interfaces for 
code intelligence tasks. The work aims to create a foundational interface that bridges visual representations with 
programmatic approaches to enhance code understanding and generation. The authors include researchers from 
University of Hong Kong, Shanghai AI Laboratory, Nanjing University, Carnegie Mellon University, and Shanghai 
Innovation Institute. The paper appears to focus on creating a novel interface paradigm for AI-assisted coding that
integrates multiple modalities.'}, {'title': 'Video-Thinker: Sparking "Thinking with Videos" via Reinforcement 
Learning', 'id': '2510.23473', 'summary': 'Video-Thinker: Sparking "Thinking with Videos" via Reinforcement 
Learning\nThis paper presents Video-Thinker, a method that enables AI systems to "think with videos" using 
reinforcement learning techniques. Building on recent advances in image reasoning, the work extends reasoning 
capabilities to video content. The authors are from Southeast University, Monash University, Xiaohongshu Inc., 
University of Southern California, and Fudan University. The paper includes code and model releases, suggesting 
it\'s a practical implementation. The approach seems to leverage RL to enhance video understanding and reasoning 
capabilities.'}, {'title': 'ReForm: Reflective Autoformalization with Prospective Bounded Sequence Optimization', 
'id': '2510.24592', 'summary': 'ReForm: Reflective Autoformalization with Prospective Bounded Sequence 
Optimization\nThis paper introduces ReForm, a method for reflective autoformalization using prospective bounded 
sequence optimization. The work focuses on the challenge of automatically formalizing mathematical content and 
reasoning. Authors are from Gaoling School of AI at Renmin University, Tongyi Lab at Alibaba Group, and Zhejiang 
University. The paper includes model, benchmark, and code releases, indicating a comprehensive approach. The method
appears to use reflection and bounded optimization to improve the autoformalization process, which is important for
mathematical reasoning and verification.'}], 'comparison': '\n=== CONSOLIDATED COMPARISON OF TOP 3 HUGGING FACE 
PAPERS ===\n\nThe three papers represent diverse but cutting-edge approaches in AI research, each targeting 
different modalities and challenges:\n\n## COMMON THEMES:\n- All three papers focus on advancing AI reasoning 
capabilities across different domains\n- Each includes practical implementations with code/model releases\n- 
Multi-institutional collaborations involving both academia and industry\n- Emphasis on foundational advances rather
than incremental improvements\n\n## DOMAIN FOCUS:\n1. **JanusCoder** (Code Intelligence): Bridges visual and 
programmatic interfaces for coding tasks\n2. **Video-Thinker** (Video Understanding): Extends reasoning from images
to video using reinforcement learning\n3. **ReForm** (Mathematical Reasoning): Tackles autoformalization challenges
in mathematics\n\n## METHODOLOGICAL APPROACHES:\n- **JanusCoder**: Multi-modal interface integration, combining 
visual and programmatic paradigms\n- **Video-Thinker**: Reinforcement learning framework for temporal reasoning in 
video\n- **ReForm**: Reflective optimization with bounded sequence planning for formalization\n\n## RESEARCH TRENDS
REPRESENTED:\nThese papers showcase three major trends in current AI research:\n1. **Multi-modal integration** 
(JanusCoder) - combining different interaction paradigms\n2. **Temporal reasoning extension** (Video-Thinker) - 
moving from static to dynamic content understanding\n3. **Mathematical rigor** (ReForm) - improving formal 
reasoning and verification capabilities\n\n## SIGNIFICANC

[Step 8: Duration 1.80 seconds| Input tokens: 35,831 | Output tokens: 3,714]

{'papers': [{'title': 'JanusCoder: Towards a Foundational Visual-Programmatic Interface for Code Intelligence',
   'id': '2510.23538',
   'summary': 'JanusCoder: Towards a Foundational Visual-Programmatic Interface for Code Intelligence\nThis paper introduces JanusCoder, a system that combines visual and programmatic interfaces for code intelligence tasks. The work aims to create a foundational interface that bridges visual representations with programmatic approaches to enhance code understanding and generation. The authors include researchers from University of Hong Kong, Shanghai AI Laboratory, Nanjing University, Carnegie Mellon University, and Shanghai Innovation Institute. The paper appears to focus on creating a novel interface paradigm for AI-assisted coding that integrates multiple modalities.'},
  {'title': 'Video-Thinker: Sparking "Thinking with Videos" via Reinforcement Learning',
   'id': '2510.23473',
   'summary': 'Video-Thinker: Sparking "Thinking with Videos" via Reinf